In [1]:
# !pip install ctransformers[cuda]
!pip3 install numpy==1.26.4
import numpy as np
np.__version__
# restart after

'1.26.4'

In [2]:
import os
# from dotenv import load_dotenv
# load_dotenv('.env')
# hf_api = os.getenv('HF_API')

In [3]:
!pip install transformers==4.38.0
!pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl
!pip install unstructured ragatouille
# reranker
from ragatouille import RAGPretrainedModel
# !nvcc --version

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
# this will take time :c
!CUDACXX=/usr/local/cuda-12.1/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 229.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 302.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 285.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 366.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 399.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.56-cp310-cp310-manylinux_2_26_x86_64.whl size=22630184 sha256=44445fb007295f0204aadaa4f723a49ffcf801940e6d2ede7292e961462b88d3
  Stored in directory: /tmp/pip-ephem-wheel-cache-5s_2jf9m/wheels/e5/09/9d/c413053f6258cb2546cc792418c595e276f9efd5db31a80377
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
  

In [7]:
# fix colab error: https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os
import pandas as pd

In [9]:
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [10]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
pd.set_option(
    "display.max_colwidth", None
)

In [11]:
# pip list --outdated

# Specify the model/versions, etc.
## also specify the knowledge base file and the reference answer directory

In [12]:
EMBEDDING_MODEL = "thenlper/gte-base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_MODEL)

In [13]:
EMBEDDING_MODEL_NAME = EMBEDDING_MODEL
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [14]:
!cd ../faiss_index_total_final
!ls

csv_qa_gold  database.py  evaluation_csv.py  Mistral-1shot.ipynb


In [18]:
FAISS_FILE = '../faiss_index_total_final_new'
# num_retrieved_docs: int = 5
# num_docs_final: int = 3
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(FAISS_FILE, embedding_model, allow_dangerous_deserialization = True)

In [19]:
model_path = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_basename = 'mistral-7b-instruct-v0.1.Q5_K_M.gguf'
model_path = hf_hub_download(repo_id=model_path, filename=model_basename)

mistral-7b-instruct-v0.1.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

# Reader LLM

In [260]:

# https://www.reddit.com/r/LocalLLaMA/comments/16v820a/mistral_7b_temperature_settings/

# llm_answer_gen = LlamaCpp(
# streaming = True,
# model_path = model_path,`
# temperature=.2,
# n_gpu_layers=-1,
# top_p=.1,
# top_k	= 40,
# repetition_penalty=	1.176,
# verbose=True,
# n_ctx=2400
# )
llm_answer_gen = LlamaCpp(
streaming = True,
model_path = model_path,
temperature = 0.7, 
n_gpu_layers = -1, 
top_p = 0.95, 
top_k = 40,
repetition_penalty = 1.100000,
verbose = True,
n_ctx = 4096, 
repeat_last_n = 64,
presence_penalty = 0.000000, 
frequency_penalty = 0.000000, 
tfs_z = 1.000000,
typical_p = 1.000000, 
mirostat = 0, 
mirostat_lr = 0.100000, 
mirostat_ent = 5.000000
);

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! repeat_last_n is not default parameter.
                repeat_last_n was transferred to model_kwargs.
                Please confirm that repeat_last_n is what you intended.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! presence_penalty is not default parameter.
                presence_penalty was transferred to model_kwargs.
                Please confirm that presence_penalty is what you intended.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! frequency_penalty is not default parameter.
                frequency_penalty was transferred to model_kwargs.
                Please confirm that frequency_penalty is what you intended.
  warnings.warn(
/home/ec2-user/a

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q5_K_M.gguf. Received error Failed to load model from file: /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q5_K_M.gguf (type=value_error)

In [21]:
llm_answer_gen.verbose = False

In [258]:
mistral = LlamaCpp(
streaming = True,
model_path = model_path,
temperature = 0.3, 
n_gpu_layers = -1, 
top_p = 0.4, 
top_k = 40,
repetition_penalty = 1.100000,
verbose = False,
n_ctx = 4096, 
);

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! repetition_penalty is not default parameter.
                repetition_penalty was transferred to model_kwargs.
                Please confirm that repetition_penalty is what you intended.
  warnings.warn(
ggml_backend_cuda_buffer_type_alloc_buffer: allocating 4807.05 MiB on device 0: cudaMalloc failed: out of memory
llama_model_load: error loading model: failed to allocate buffer
llama_load_model_from_file: failed to load model


ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q5_K_M.gguf. Received error Failed to load model from file: /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q5_K_M.gguf (type=value_error)

In [259]:
mistral_new = LlamaCpp(
streaming = True,
model_path = model_path,
temperature = 0.5, 
n_gpu_layers = -1, 
top_p = 0.4, 
top_k = 40,
repetition_penalty = 1.100000,
verbose = False,
n_ctx = 4096, 
);

ggml_backend_cuda_buffer_type_alloc_buffer: allocating 4807.05 MiB on device 0: cudaMalloc failed: out of memory
llama_model_load: error loading model: failed to allocate buffer
llama_load_model_from_file: failed to load model


ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q5_K_M.gguf. Received error Failed to load model from file: /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q5_K_M.gguf (type=value_error)

# RAG Function

In [252]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# https://www.promptingguide.ai/models/mistral-7b

def answer_mistral(
    question: str,
    knowledge_index: FAISS,
    reranker: RAGPretrainedModel,
    num_retrieved_docs: int = 10,
    num_docs_final: int = 3,
    llm: LlamaCpp = llm_answer_gen,
    ): #-> Tuple[str, List[LangchainDocument]]

    print("=> Retrieving documents...")
    relevant_docs_acquired = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_doc_id = [doc.metadata['source'] for doc in relevant_docs_acquired]

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = [doc.page_content for doc in relevant_docs_acquired]
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)

        relevant_docs = [LangchainDocument(page_content=doc["content"],
                                           metadata={'source': relevant_doc_id[ doc['result_index']] } ) for doc in relevant_docs]

        relevant_docs_content = [doc.page_content for doc in relevant_docs]
        relevant_doc_id = [doc.metadata for doc in relevant_docs]

    else:
        relevant_docs_content = [doc.page_content for doc in relevant_docs_acquired]
        relevant_docs = relevant_docs_acquired

    relevant_docs_content = relevant_docs_content[:num_docs_final]
    relevant_docs_indexed = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"\nDocument {str(i)}:::\n\n" + doc for i, doc in enumerate(relevant_docs_content)])

    # context_and_question = f"Please provide concise answers. Extract the most relevant information from the given context while keeping the responses as short as possible. Include multiple correct answers if necessary. If no relevant context, then say 'I dont know'.\n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAgain we just want main key points in the answers. For example, if the question is 'What is the birthday of Gandhi?'; the answer should be 'October 2, 1869'\n.Answer:"
    
    context_and_question = f"""[INST] Given the context: 
    {context}

    Answer the following question concisely using keywords/key information from the context:
    {question}
    
    Here are the guidelines for answering:
    - Identify relevant details to answer the query
    - Try to just give the answer without repeating words from the question in the answer
    - The answer is simply name, place, time, unless asked for more complicated questions
    - Provide a short, precise answer with keywords only; the shorter and more accurate the answer the better
    - Do not repeat a version of the question in the answer.
    
    Now answer the questions based on the context, query, and the guidelines.
    [/INST]"""
    
    # print(context_and_question)
    answer = llm.invoke(context_and_question)
    if answer is not None:
        new_answer = answer.split(': ', 1)
        if len(new_answer) > 1:
            answer = new_answer[1]
        else:
            answer = new_answer[0]
    # print(f"Question: {question}")
    forbidden_list = ["According to the provided context, ", "Based on the provided context,", '\n']
    for phrase in forbidden_list:
        answer = answer.replace(phrase, " ")
    answer = answer.replace("<pad>", "").replace("</s>", "").replace("\n", "").strip()
    # print(f"Answer: {answer}")
    return answer, relevant_docs_content

In [253]:
question = "How should Kilties promote the success of the Kiltie Band?"
answer, docs = answer_mistral(question, KNOWLEDGE_VECTOR_DATABASE,
                            reranker=RERANKER, llm = llm_answer_gen)
answer

=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 283.4 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 12.58it/s]
Llama.generate: prefix-match hit


'To promote the success of the Kiltie Band, Kilties should attend rehearsals, perform at games, and participate in events such as the holiday concert and spring concerts. They should also promote the band through word-of-mouth and social media. Additionally, they can volunteer to help with fundraising efforts and support other activities related to the band.'

----

In [257]:
question = "What is the ranking of the School of Computer Science?"
answer, docs = answer_mistral(question, KNOWLEDGE_VECTOR_DATABASE,
                            reranker=RERANKER, llm = mistral)
answer

=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 584.9 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 11.70it/s]


'The School of Computer Science at Carnegie Mellon University is ranked with the best graduate programs in the world by a leading U.S. magazine and its undergraduate programs are also rated the best in the U.S. by corporate recruiters.'

## Evaluation

In [231]:
# specify output csv 
input_file = 'qa_gold-mistral-shot-0shot.csv'
output_file = 'mistral-shot-0shot-output.csv'
csv_input_dir = f'csv_qa_gold/{input_file}'
csv_output_dir = f'csv_qa_gold/{output_file}'

In [232]:
def initialize_model_answer_column(df):
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = None
    return df

In [233]:
import glob
import os
import pandas as pd

directory_csv = 'csv_qa_gold'
csv_files = ['csv_qa_gold/test_combined.csv']

# read in the csv files in the directory and concatenate
df_total = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    print(df.shape)
    df_total = pd.concat([df_total, df], axis=0)
print(df_total.shape)
df_total = initialize_model_answer_column(df_total)
df_total.to_csv(csv_input_dir, index=False) 
# doing this so the original is untouched

(191, 3)
(191, 3)


In [234]:
# df_total.iloc[10:20, :] # please check that excel does not fuck up for row 15,16 the November 2006 to Nov-06 or some other format

In [235]:
def generate_answer(question,llm, return_relevant_docs = False):
    answer, relevant_docs = answer_mistral(question, KNOWLEDGE_VECTOR_DATABASE,
                            reranker=RERANKER, llm = llm)
    if return_relevant_docs:
        return answer, relevant_docs
    return answer

In [236]:
# Function to run over a list of questions and return the answers
# Define the function to process the CSV and add model answers
def generate_answers_and_save(csv_path, llm):
    df = pd.read_csv(csv_path)
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = ''
    for index, row in df.iterrows():
        ModelAnswer = generate_answer(row['Question'],llm=llm)
        print(ModelAnswer)
        df.at[index, 'ModelAnswer'] = ModelAnswer
        df.to_csv(csv_output_dir, index=False)
    return df

In [237]:
df_with_model_answers = generate_answers_and_save(csv_path=csv_input_dir, llm = mistral)
print(df_with_model_answers.shape)
df_with_model_answers.head()

=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 16.46it/s]


No, students in Kiltie do not have to memorize music.
=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 16.31it/s]


No, you do not need to email your home address to the Kiltie Band Director if you're interested in joining.
=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


...

100%|██████████| 1/1 [00:00<00:00, 82.91it/s]


Justine Cassell
=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 68.67it/s]


Lori S Levin
(191, 4)


,Category,Question,Answer,ModelAnswer
0,webpages,Do students in Kiltie have to memorize music?,No,"No, students in Kiltie do not have to memorize music."
1,webpages,Do I need to email my home address to the Kiltie Band Director if I'm interested in joining?,Yes,"No, you do not need to email your home address to the Kiltie Band Director if you're interested in joining."
2,webpages,Do I need to audition to join the Kiltie Band?,No,No audition is required to join the Kiltie Band.
3,webpages,Where does the Kiltie Band rehearse?,CUC Studio Theater,Where does the Kiltie Band rehearse? The Kiltie Band rehearses in the CUC Studio Theater.
4,webpages,When should I get an instrument from the Kiltie Band if I need one?,4:30p.m. before the first rehearsal,"To get an instrument from the Kiltie Band, you should visit the facilities during their designated hours and make a reservation if necessary. If you need an instrument during the football season, you should visit the facilities before the first rehearsal on Monday at 5:30 p.m. in the CUC Studio Theater. If you need an instrument during the concert season, you should visit the facilities on Mondays and Thursdays from 5:30-6:30 p.m. in the CUC Studio Theater."


In [238]:
# with additional extraction prompt after generation without prompt
type(df_with_model_answers['ModelAnswer'][0]),type(df_with_model_answers['Answer'][0])
df_with_model_answers = df_with_model_answers.astype(str)  # Convert columns to string type
df_with_model_answers = df_with_model_answers[df_with_model_answers['Category'] != 'json_hard']

from evaluation_csv import total_score_csv
exact_match, f1_score, recall_score = total_score_csv(df_with_model_answers['ModelAnswer'], df_with_model_answers['Answer'])
print(f'Exact match: {exact_match}')
print(f'F1 score: {f1_score}')
print(f'Recall score: {recall_score}')

Exact match: 0.022099447513812154
F1 score: 0.2344548582518731
Recall score: 0.5605947865684056


In [239]:
for category in df_with_model_answers['Category'].unique():
    print(f'Category: {category}')
    df_category = df_with_model_answers[df_with_model_answers['Category'] == category]
    exact_match, f1_score, recall_score = total_score_csv(df_category['ModelAnswer'], df_category['Answer'])
    print(f'Exact match: {exact_match}')
    print(f'F1 score: {f1_score}')
    print(f'Recall score: {recall_score}')
    print('\n')

Category: webpages
Exact match: 0.0
F1 score: 0.23341955615259294
Recall score: 0.6972235759150963


Category: tabular_webpages
Exact match: 0.0
F1 score: 0.1391306868420874
Recall score: 0.4286057692307692


Category: other_pdf
Exact match: 0.0
F1 score: 0.22223822936049614
Recall score: 0.4265010351966874


Category: papers_pdf
Exact match: 0.0
F1 score: 0.07513518227803942
Recall score: 0.22222222222222224


Category: schedule_pdf
Exact match: 0.0
F1 score: 0.09292929292929294
Recall score: 0.13636363636363635


Category: jsons
Exact match: 0.05714285714285714
F1 score: 0.2992276364059385
Recall score: 0.6299258799527787




---

In [ ]:
# specify output csv 
input_file = 'qa_gold-mistral-shot-1shot.csv'
output_file = 'mistral-reddit-output.csv'
csv_input_dir = f'csv_qa_gold/{input_file}'
csv_output_dir = f'csv_qa_gold/{output_file}'

import glob
import os
import pandas as pd

directory_csv = 'csv_qa_gold'
csv_files = ['csv_qa_gold/test_combined.csv']

# read in the csv files in the directory and concatenate
df_total = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    print(df.shape)
    df_total = pd.concat([df_total, df], axis=0)
print(df_total.shape)
df_total = initialize_model_answer_column(df_total)
df_total.to_csv(csv_input_dir, index=False) 
# doing this so the original is untouched

In [ ]:
df_with_model_answers = generate_answers_and_save(csv_path=csv_input_dir, llm = llm_answer_gen)
print(df_with_model_answers.shape)
df_with_model_answers.head()

In [ ]:
# with additional extraction prompt after generation without prompt
type(df_with_model_answers['ModelAnswer'][0]),type(df_with_model_answers['Answer'][0])
df_with_model_answers = df_with_model_answers.astype(str)  # Convert columns to string type
df_with_model_answers = df_with_model_answers[df_with_model_answers['Category'] != 'json_hard']

from evaluation_csv import total_score_csv
exact_match, f1_score, recall_score = total_score_csv(df_with_model_answers['ModelAnswer'], df_with_model_answers['Answer'])
print(f'Exact match: {exact_match}')
print(f'F1 score: {f1_score}')
print(f'Recall score: {recall_score}')

In [ ]:
for category in df_with_model_answers['Category'].unique():
    print(f'Category: {category}')
    df_category = df_with_model_answers[df_with_model_answers['Category'] == category]
    exact_match, f1_score, recall_score = total_score_csv(df_category['ModelAnswer'], df_category['Answer'])
    print(f'Exact match: {exact_match}')
    print(f'F1 score: {f1_score}')
    print(f'Recall score: {recall_score}')
    print('\n')